Package Installations

In [1]:
!pip install kaggle

In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 15.3 MB/s 
     |████████████████████████████████| 77 kB 8.3 MB/s 
     |████████████████████████████████| 6.6 MB 43.4 MB/s 
     |████████████████████████████████| 596 kB 65.9 MB/s 
     |████████████████████████████████| 895 kB 50.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install -U torchtext==0.8.0

     |████████████████████████████████| 6.9 MB 15.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [4]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import random
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm.notebook import tqdm
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import transformers
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
seed_val = 500
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Kaggle Dataset

In [7]:
# Create the kaggle directory and read the uploaded kaggle.json file
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 600 /root/.kaggle/kaggle.json

In [9]:
# Download dataset
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

100% 41.0M/41.0M [00:00<00:00, 83.9MB/s]
100% 41.0M/41.0M [00:00<00:00, 61.0MB/s]


In [10]:
# Unzip folder in Colab content folder
!unzip /content/fake-and-real-news-dataset.zip

Archive:  /content/fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [11]:
df1 = pd.read_csv("Fake.csv")
df2 = pd.read_csv("True.csv")
df1["y"] = ["False"]*len(df1)
df2["y"] = ["True"]*len(df2)

In [12]:
frames = [df1, df2]
corpus = pd.concat(frames, ignore_index=True)

In [13]:
corpus["x"] = corpus["title"]

In [14]:
y_encoder = LabelEncoder()
corpus['y'] = y_encoder.fit_transform(corpus['y'])

In [15]:
corpus = corpus[["x", "y"]]

In [16]:
corpus.drop_duplicates(inplace=True)
corpus

,x,y
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0
...,...,...
44892,North Korea shipments to Syria chemical arms a...,1
44894,LexisNexis withdrew two products from Chinese ...,1
44895,Minsk cultural hub becomes haven from authorities,1
44896,Vatican upbeat on possibility of Pope Francis ...,1


**Reddit Dataset**

In [17]:
reddit_df = pd.read_csv('/content/news_posts.csv')
reddit_df

,Unnamed: 0,Title,Post Text,ID,Score,Total Comments,Post URL
0,0,People Are Accusing Robinhood Of Stealing From...,NaN,l7afyx,181017,4408,https://www.buzzfeednews.com/article/clarissaj...
1,1,US Military Could Lose Space Force Trademark t...,NaN,gyzw2p,129257,2844,https://www.cbr.com/us-military-lose-space-for...
2,2,White House threatens to fire anyone who tries...,NaN,jrskag,126468,4141,https://americanindependent.com/white-house-th...
3,3,Meta's threat to close down Facebook and Insta...,NaN,so0ree,126258,3786,https://www.cityam.com/metas-threat-to-close-d...
4,4,Don't eat or inject yourself with disinfectant...,NaN,g6zci5,125437,7024,https://www.cnn.com/world/live-news/coronaviru...
...,...,...,...,...,...,...,...
4898,987,Colorado pizza delivery driver saves mans life...,NaN,3b37gf,24221,898,http://www.postindependent.com/news/16943384-1...
4899,988,"Utah Woman Donates Over 1,200 Handmade Toys to...",NaN,90pbng,24216,888,https://www.insideedition.com/utah-woman-donat...
4900,989,Year of the Tiger marks increase in tiger popu...,NaN,sinyqg,24197,205,https://democratic-europe.eu/2022/02/01/%ef%bf...
4901,990,Florida cops deliver dresses made by a 99-year...,NaN,8q8njr,24164,183,http://www.miamiherald.com/news/state/florida/...


In [18]:
reddit_df['x'] = reddit_df['Title']

In [19]:
reddit_df['y'] = ["True"]*4903
reddit_df['y'] = y_encoder.transform(reddit_df['y'])

In [20]:
reddit_df = reddit_df[["x", "y"]]
reddit_df.drop_duplicates(inplace=True)
reddit_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,People Are Accusing Robinhood Of Stealing From...,1
1,US Military Could Lose Space Force Trademark t...,1
2,White House threatens to fire anyone who tries...,1
3,Meta's threat to close down Facebook and Insta...,1
4,Don't eat or inject yourself with disinfectant...,1
...,...,...
4898,Colorado pizza delivery driver saves mans life...,1
4899,"Utah Woman Donates Over 1,200 Handmade Toys to...",1
4900,Year of the Tiger marks increase in tiger popu...,1
4901,Florida cops deliver dresses made by a 99-year...,1


**Twitter Dataset**

In [21]:
twitter_real_df = pd.read_csv('/content/real_news_tweets.csv')
twitter_real_df

,Unnamed: 0,Datetime,Tweet Id,Text,Username
0,0,2022-04-19 08:35:25+00:00,1516334641610989571,Half-a-million Ukrainians have fled to the Lvi...,AP
1,1,2022-04-19 07:42:46+00:00,1516321393281015809,Afghan police say explosions targeting educati...,AP
2,2,2022-04-19 07:13:55+00:00,1516314133666349058,Russian forces have attacked along a broad fro...,AP
3,3,2022-04-19 06:40:38+00:00,1516305757674819584,There's stark public divide in Iran over Russi...,AP
4,4,2022-04-19 06:26:26+00:00,1516302180772364288,There's stark public divide in Iran over Russi...,AP
...,...,...,...,...,...
4995,4995,2022-02-24 00:13:22+00:00,1496639353820430337,The soldiers on the front line of the Ukraine-...,BBCWorld
4996,4996,2022-02-23 23:45:08+00:00,1496632246459158531,What's Putin's next move?,BBCWorld
4997,4997,2022-02-23 23:29:06+00:00,1496628212880875520,"Virgin Hyperloop, a futuristic train service e...",BBCWorld
4998,4998,2022-02-23 23:14:06+00:00,1496624437243879429,Metaverse app allows kids into virtual strip c...,BBCWorld


In [22]:
twitter_fake_df = pd.read_csv('/content/fake_news_tweets.csv')
twitter_fake_df

,Unnamed: 0,Datetime,Tweet Id,Text,Username
0,0,2022-04-19 08:30:02+00:00,1516333287744102405,Grandpa Surprisingly Willing To Talk About Man...,TheOnion
1,1,2022-04-19 06:30:01+00:00,1516303084628099076,Nation Attempts To Fall Asleep By Doing Little...,TheOnion
2,2,2022-04-19 05:00:01+00:00,1516280436334473221,"When it comes to pip blips, Megan Fox takes th...",TheOnion
3,3,2022-04-19 04:30:01+00:00,1516272886314672130,Longtime Sleepytime Tea Addict Has To Use 6 Ba...,TheOnion
4,4,2022-04-19 03:15:00+00:00,1516254005869096968,The Onion: Now on another part of your phone. ...,TheOnion
...,...,...,...,...,...
4995,4995,2021-10-02 14:00:00+00:00,1444301140703780868,Boss Warns All New Employees He Will Fire Them...,TheBabylonBee
4996,4996,2021-10-02 13:00:02+00:00,1444286051204403205,10 Tips For Making Sure Your Kids Are Terrifie...,TheBabylonBee
4997,4997,2021-10-02 12:21:32+00:00,1444276362865479696,Biden Says We Can Afford $3.5T Bill Because Ch...,TheBabylonBee
4998,4998,2021-10-02 02:00:00+00:00,1444119947274334211,Fly That Landed On Mike Pence During Debate Re...,TheBabylonBee


In [23]:
twitter_real_df['x'] = twitter_real_df['Text']
twitter_fake_df['x'] = twitter_fake_df['Text']

In [24]:
twitter_real_df['y'] = ['True'] * 5000
twitter_real_df['y'] = y_encoder.transform(twitter_real_df['y'])

In [25]:
twitter_fake_df['y'] = ['False'] * 5000
twitter_fake_df['y'] = y_encoder.transform(twitter_fake_df['y'])

In [26]:
twitter_real_df = twitter_real_df[["x", "y"]]
twitter_real_df.drop_duplicates(inplace=True)
twitter_real_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,Half-a-million Ukrainians have fled to the Lvi...,1
1,Afghan police say explosions targeting educati...,1
2,Russian forces have attacked along a broad fro...,1
3,There's stark public divide in Iran over Russi...,1
4,There's stark public divide in Iran over Russi...,1
...,...,...
4995,The soldiers on the front line of the Ukraine-...,1
4996,What's Putin's next move?,1
4997,"Virgin Hyperloop, a futuristic train service e...",1
4998,Metaverse app allows kids into virtual strip c...,1


In [27]:
twitter_fake_df = twitter_fake_df[["x", "y"]]
twitter_fake_df.drop_duplicates(inplace=True)
twitter_fake_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,Grandpa Surprisingly Willing To Talk About Man...,0
1,Nation Attempts To Fall Asleep By Doing Little...,0
2,"When it comes to pip blips, Megan Fox takes th...",0
3,Longtime Sleepytime Tea Addict Has To Use 6 Ba...,0
4,The Onion: Now on another part of your phone. ...,0
...,...,...
4995,Boss Warns All New Employees He Will Fire Them...,0
4996,10 Tips For Making Sure Your Kids Are Terrifie...,0
4997,Biden Says We Can Afford $3.5T Bill Because Ch...,0
4998,Fly That Landed On Mike Pence During Debate Re...,0


In [28]:
from sklearn import model_selection
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [29]:
frame1 = [twitter_real_df, twitter_fake_df]
corpus_tweet = pd.concat(frame1)
corpus_tweet

,x,y
0,Half-a-million Ukrainians have fled to the Lvi...,1
1,Afghan police say explosions targeting educati...,1
2,Russian forces have attacked along a broad fro...,1
3,There's stark public divide in Iran over Russi...,1
4,There's stark public divide in Iran over Russi...,1
...,...,...
4995,Boss Warns All New Employees He Will Fire Them...,0
4996,10 Tips For Making Sure Your Kids Are Terrifie...,0
4997,Biden Says We Can Afford $3.5T Bill Because Ch...,0
4998,Fly That Landed On Mike Pence During Debate Re...,0


**BiLSTM**

In [30]:
x_train =corpus['x']
y_train = corpus['y']
x_test = pd.concat([corpus_tweet['x'], reddit_df['x']])
y_test = pd.concat([corpus_tweet['y'], reddit_df['y']])

In [31]:
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 750 # max number of words in a question to use
batch_size = 512 # how many samples to process at once
n_epochs = 5 # how many times to iterate over all samples
n_splits = 5 # Number of K-fold Splits
SEED = 10
debug = 0

In [32]:
## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train))
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

## Pad the sentences 
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test= pad_sequences(x_test, maxlen=maxlen)

In [33]:
print("X_test: ",x_train, x_train.shape, x_train.shape[1])
print("X_test: ",x_test)

X_test:  [[    0     0     0 ...    43    15  1771]
 [    0     0     0 ...    87  1806   379]
 [    0     0     0 ...  7193     8 11526]
 ...
 [    0     0     0 ...  6253    22  2434]
 [    0     0     0 ...  3112  3557    38]
 [    0     0     0 ...     4    87  2404]] (38729, 750) 750
X_test:  [[   0    0    0 ...   15   11  582]
 [   0    0    0 ...   11 3951 4154]
 [   0    0    0 ...   17  577 1363]
 ...
 [   0    0    0 ...    3  922  301]
 [   0    0    0 ... 6656    8 2375]
 [   0    0    0 ... 1327    3 2283]]


In [34]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!ls -lat

--2022-04-24 05:23:47--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-04-24 05:23:47--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-04-24 05:23:47--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [35]:
## FUNCTIONS TAKEN FROM https://www.kaggle.com/gmhost/gru-capsule

def load_glove(word_index):
    EMBEDDING_FILE = '/content/glove.6B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [36]:
if debug:
    embedding_matrix = np.random.randn(120000,300)
else:
    embedding_matrix = load_glove(tokenizer.word_index)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
np.shape(embedding_matrix)

(31803, 300)

TextCNN

In [38]:
import torch
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch as t

#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
import os 

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

In [39]:
class TextCNN_Classifier(nn.Module):
    
    def __init__(self):
        super(TextCNN_Classifier, self).__init__()
        #kernel size will be filter _size * embedding size
        # we will have 5 filter covering these many words at a time
        self.filter_1 = 1 
        self.filter_2 = 2
        self.filter_3 = 3
        self.filter_4 = 4
        self.filter_5 = 5
        num_filters = 15 # no of output channels
        number_classes = 2
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.conv_1 = nn.Conv2d(1,num_filters,(self.filter_1, embed_size))
        self.conv_2 = nn.Conv2d(1,num_filters,(self.filter_2, embed_size))
        self.conv_3 = nn.Conv2d(1,num_filters,(self.filter_3, embed_size))
        self.conv_4 = nn.Conv2d(1,num_filters,(self.filter_4, embed_size))
        self.conv_5 = nn.Conv2d(1,num_filters,(self.filter_5, embed_size))
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(5*num_filters, number_classes) # number of layers is 5

    def forward(self, x):
        x = self.embedding(x)  
        x =  x.unsqueeze(1)
        x1 =  F.max_pool1d( F.relu(self.conv_1(x)).squeeze(3) , F.relu(self.conv_1(x)).squeeze(3).size(2)).squeeze(2)
        x2 =  F.max_pool1d( F.relu(self.conv_2(x)).squeeze(3) , F.relu(self.conv_2(x)).squeeze(3).size(2)).squeeze(2)
        x3 =  F.max_pool1d( F.relu(self.conv_3(x)).squeeze(3) , F.relu(self.conv_3(x)).squeeze(3).size(2)).squeeze(2)
        x4 =  F.max_pool1d( F.relu(self.conv_4(x)).squeeze(3) , F.relu(self.conv_4(x)).squeeze(3).size(2)).squeeze(2)
        x5 =  F.max_pool1d( F.relu(self.conv_5(x)).squeeze(3) , F.relu(self.conv_5(x)).squeeze(3).size(2)).squeeze(2)
        x = torch.cat((x1,x2,x3,x4,x5),1)
        x = self.dropout(x)
        x = self.fc1(x) 
        return x

In [40]:
n_epochs = 10
model = TextCNN_Classifier()
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
model.cuda()

# Load train and test in CUDA Memory
x_train_2 = torch.tensor((x_train), dtype=torch.long).cuda()
y_train_2 = torch.tensor(y_train.values, dtype=torch.long).cuda()
x_cv = torch.tensor(x_test, dtype=torch.long).cuda()
y_cv = torch.tensor(y_test.values, dtype=torch.long).cuda()

# Create Torch datasets
train = torch.utils.data.TensorDataset(x_train_2, y_train_2)
valid = torch.utils.data.TensorDataset(x_cv, y_cv)

# Create Data Loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

train_loss = []
valid_loss = []

for epoch in range(n_epochs):
    # Set model to train configuration
    model.train()
    avg_loss = 0.  
    for i, (x, y) in enumerate(train_loader):
        # Predict/Forward Pass
        y_pred = model(x)
        # Compute loss
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)
    
    # Set model to validation configuration -Doesn't get trained here
    model.eval()        

    val_preds = np.zeros((len(x_cv),2))
    #y_pred_2 =[]
    y_true =[]
    preds = []
    labels = []
    for i, (x, y) in enumerate(valid_loader):
      y_pred_2=torch.argmax(model(x), dim = 1).tolist()
      y_pred_2 = map(int, y_pred_2)
      preds.extend(list(y_pred_2))
      labels.extend(y.tolist())
    
    val_accuracy = (np.sum(np.array(preds) == np.array(labels)))/len(preds)
    f1_scorE_sklearn = f1_score(preds,labels,average="weighted")
    print("f1 score is: ")
    print(f1_scorE_sklearn)
    print('Epoch {}/{} \t val_acc={:.4f} \t f1_score={:.4f}'.format(
                epoch + 1, n_epochs, val_accuracy, f1_scorE_sklearn))


f1 score is: 
0.5551328576549218
Epoch 1/10 	 val_acc=0.5689 	 f1_score=0.5551
f1 score is: 
0.5277236715650553
Epoch 2/10 	 val_acc=0.5423 	 f1_score=0.5277
f1 score is: 
0.5017231935162019
Epoch 3/10 	 val_acc=0.5128 	 f1_score=0.5017
f1 score is: 
0.5077004588470493
Epoch 4/10 	 val_acc=0.5232 	 f1_score=0.5077
f1 score is: 
0.5092857729366013
Epoch 5/10 	 val_acc=0.5256 	 f1_score=0.5093
f1 score is: 
0.5000738697041491
Epoch 6/10 	 val_acc=0.5161 	 f1_score=0.5001
f1 score is: 
0.49725298237161536
Epoch 7/10 	 val_acc=0.5109 	 f1_score=0.4973
f1 score is: 
0.4979358244895236
Epoch 8/10 	 val_acc=0.5117 	 f1_score=0.4979
f1 score is: 
0.49688608381071525
Epoch 9/10 	 val_acc=0.5120 	 f1_score=0.4969
f1 score is: 
0.49163055701717223
Epoch 10/10 	 val_acc=0.5052 	 f1_score=0.4916
